In [1]:
import re
import os
import pysolr
from datetime import datetime
from pprint import pprint

In [2]:
# Solr collection url 
SOLR_URL = 'http://solr:8983/solr/cmp269'

QUERIES_FILE_PATH = 'data/Consultas.txt'

In [3]:
def parse_doc(doc_as_string):
    # print(doc_as_string)

    doc_tpl = {
        'num': 'num',
        'PT-title': 'title',
        'PT-desc': 'desc',
        'PT-narr': 'narr',
    }

    doc = {}

    for key in doc_tpl.keys():
        # generates something like:
        # str_pattern = ".*\<DOCID\>(.*)\<\/DOCID\>.*"
        str_pattern = ".*\<%s\>(.*)\<\/%s\>.*" % (key, key)

        # compiles pattern, capture match, updates doc
        re_pattern = re.compile(str_pattern, re.DOTALL)
        match = re_pattern.match(doc_as_string)
        
        if match:
            value = match.group(1).strip()
            solr_key = doc_tpl[key]
            doc.update({solr_key: value})

    # ensures we have all keys..
    for key in doc_tpl.values():
        assert(doc[key] is not None)

    return doc


In [4]:
def parse_file(fpath, doc_tag):
    '''Reads file line by line and extract the docs'''

    docs = []
    closing_doc_line = "</%s>\n" % doc_tag
    
    with open(fpath, encoding="iso-8859-1") as file:
        doc_as_string = ""

        line = file.readline()
        while (line):
            if (line) == closing_doc_line:
                doc_as_string += line
                
                doc = parse_doc(doc_as_string)
                docs.append(doc)

                doc_as_string = ""
            else:
                doc_as_string += line

            line = file.readline()

    return docs

In [5]:
queries = parse_file(QUERIES_FILE_PATH, 'top')
sq = pysolr.Solr(SOLR_URL)

for query in queries:
    title = query["title"]

    fq = [
        'text_txt_pt:"%s"' % title
    ]
    
    # runs the query
    result = sq.search(q="*:*", fq=fq)
    
    print("Encontrados %d resultados para a busca '%s'" % (result.hits, title))    
    [ print(d["docno_s"]) for d in result.docs ]

    result = None

print("The end")

Encontrados 2 resultados para a busca 'Medicina alternativa'
FSP950315-114
FSP950924-010
Encontrados 0 resultados para a busca 'Sistemas de reforma e pensões na Europa'
Encontrados 0 resultados para a busca 'Países com pena de morte'
Encontrados 0 resultados para a busca 'Danos provocados por terramotos'
Encontrados 0 resultados para a busca 'Viciados na Internet'
Encontrados 0 resultados para a busca 'Doença de Creutzfeldt-Jakob'
Encontrados 0 resultados para a busca 'Limpeza étnica nos Balcãs'
Encontrados 0 resultados para a busca 'Legislação anti-tabagista'
Encontrados 12 resultados para a busca 'Adivinhação'
FSP950518-098
FSP950521-035
FSP950308-003
FSP950314-124
FSP950109-006
FSP950221-007
FSP950728-121
FSP951018-148
FSP951013-101
FSP950915-120
Encontrados 0 resultados para a busca 'Espectáculos de beneficiência'
Encontrados 0 resultados para a busca 'Disputas de arbitragem no futebol'
Encontrados 0 resultados para a busca 'Contrabando de material radioativo'
Encontrados 0 resulta